In [133]:
import os
import difflib
from tqdm import tqdm
import pandas as pd
import re
import sys
import json
import base64
import hashlib
import requests
from typing import Union
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config

In [4]:
auth = HTTPBasicAuth(config['user_1C'], config['password_1C'])

### Получение списка номеров счетов

In [5]:
# Получение списка номеров счетов за период времени

# месяцы 04-06 уже есть
month1 = '07'
month2 = '08'

date_1 = f'01-{month1}-2024'
date_2 = f'01-{month2}-2024'

deal_numbers_list = requests.get(fr'http://10.10.0.10:81/ca/hs/interaction/InvoicesByDate/{date_1}/{date_2}', auth=auth).json()
print(f"length: {len(deal_numbers_list)}")

print(f"deal_numbers: {deal_numbers_list[0:3]}")

length: 2002
deal_numbers: ['0С000000000000137236', '0С000000000000138343', '0С000000000000138344']


### Получение деталей по данным счетам

```
[
{"СтрокиСчета": ["Услуга": , "УслугаКод": , "ТранспортнаяПозиция": , "Количество": , "Цена: "], "ФайлыСчета": [..]},
{"СтрокиСчета": [..], "ФайлыСчета": [..]},
..
]
```

In [ ]:
# Получение деталей счетов
# ЗДЕСЬ ДОБАВИТЬ НОМЕР СЧЕТА К КАЖДОМУ СЛОВАРЮ ДЕТАЛЕЙ

numbers = []
for number in tqdm(deal_numbers_list):
    try:
        res = requests.get(fr'http://10.10.0.10:81/ca/hs/interaction/InvoiceDetailsByNumber/{number}', auth=auth).json()
    except:
        print(number)
        continue
    numbers.append(res)

In [ ]:
numbers[0]

In [ ]:
# Запись деталей в файл

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', f'result_numbers_{month1}.json')
with open(file, 'w', encoding='utf-8') as f:
    json.dump(numbers, f, ensure_ascii=False, indent=4)

### Фильтрация деталей

In [7]:
# Получение списка частых услуг

file = os.path.join(config['BASE_DIR'], 'config', 'freq_services.json')
with open(file, 'r', encoding='utf-8') as f:
    freq_services = json.load(f)


def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)
    
freq_services = [remove_special_characters(service) for service in freq_services]

In [8]:
# Определение функций очистки

# hash

def calculate_hash(file_path):
    # Инициализация хэш-объекта MD5
    md5_hash = hashlib.md5()

    # Открываем файл в бинарном режиме для чтения
    with open(file_path, "rb") as f:
        # Чтение файла блоками по 4096 байт (можно настроить)
        for byte_block in iter(lambda: f.read(4096), b""):
            md5_hash.update(byte_block)

    # Возвращаем хэш-сумму в виде шестнадцатеричной строки
    return md5_hash.hexdigest()


def is_single_hash(lst: list[str]) -> str | None:
    """
    lst: result['ФайлыСчета'] from result = requests.get(fr'http://.../InvoiceDetailsByNumber/{deal}', auth=auth).json()
    """
    if not list:
        return None
        
    res = [calculate_hash(file) for file in lst]
        
    if len(set(res)) == 1:
        return lst[0]


# frequent services

def is_all_services_freq(lst: list, freq_services: list) -> bool:
    """
    lst: result['СтрокиСчета'] from result = requests.get(fr'http://.../InvoiceDetailsByNumber/{deal}', auth=auth).json()
    freq_services: list of frequently used services (cleared)
    """
    services = [d['Услуга'] for d in lst]
    clear_services = [remove_special_characters(service) for service in services]
    
    if set(clear_services).issubset(freq_services):
        return True
    else:
        return False    

In [134]:
# неотфильтрованные детали

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', 'result_numbers_07.json')
with open(file, 'r', encoding='utf-8') as f:
    result = json.load(f)

print(len(result))

1963


In [ ]:
# отфильтрованные детали

filtered_response_list = []
for res in tqdm(result):
    
    raws = res['СтрокиСчета']
    files = res['ФайлыСчета']
    
    if is_all_services_freq(raws, freq_services):
        # print('freq')
        continue
    if not is_single_hash(files):
        # print('hash')
        continue

    filtered_response_list.append(res)

print(f"{len(deal_numbers_list)} --> {len(filtered_response_list)}")

In [11]:
filtered_response_list[0]

{'СтрокиСчета': [{'Услуга': 'Терминальная обработка',
   'УслугаКод': '000000421',
   'ТранспортнаяПозиция': 'CICU9764876',
   'Количество': 1,
   'Цена': 10500}],
 'ФайлыСчета': ['\\\\10.10.0.3\\docs\\Baltimpex\\Invoice\\Import\\АА-0093285\\T1_ОФ-00011348_71c5acb7-b8f5-4cda-b618-bd631ff0c088_СчетНаОплату.pdf']}

In [12]:
# Запись отфильтрованных деталей в файл

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', f'filtered_result_numbers_{month1}.json')
with open(file, 'w', encoding='utf-8') as f:
    json.dump(filtered_response_list, f, ensure_ascii=False, indent=4)

In [36]:
# запись предварительно отформатированных услуг в формате
# Услуга | УслугаКод | Цена

with open(os.path.join(config['BASE_DIR'], 'config', 'new_services.txt'), 'a', encoding='utf-8') as file:
    for i, res in enumerate(result):
        print(f'-----{i}-----')
        raws = res['СтрокиСчета']
        files = res['ФайлыСчета']
    
        for raw in raws:
            service = raw['Услуга']
            code = raw['УслугаКод']
            price = raw['Цена']
            s = f"{service}|{code}|{price}\n"
            print(s.strip())
            if i > 57:
                file.write(s)
        print('file:')
        print(files[0])

-----0-----
Терминальная обработка|000000421|10500
file:
\\10.10.0.3\docs\Baltimpex\Invoice\Import\АА-0093285\T1_ОФ-00011348_71c5acb7-b8f5-4cda-b618-bd631ff0c088_СчетНаОплату.pdf
-----1-----
Линейный фрахтовый сбор (THC)|000000420|39410.24
выдача документов|ТК-008754|4529.92
Оформление релиза (конт.)|ТК-008755|4529.92
file:
\\10.10.0.3\docs\Baltimpex\Invoice\Import\АА-0093442\Модуль_24-24712_24-24712 ООО Содружество Балтика-Транс.doc
-----2-----
Комплекс услуг по прибытию 20' контейнера на ст. Раменское|ТК-008899|9500
Распечатывание товаросопроводительных документов|ТК-009018|200
Комплекс услуг по прибытию 20' контейнера на ст. Раменское|ТК-008899|9500
Распечатывание товаросопроводительных документов|ТК-009018|200
Комплекс услуг по прибытию 20' контейнера на ст. Раменское|ТК-008899|9500
Распечатывание товаросопроводительных документов|ТК-009018|200
file:
\\10.10.0.3\docs\Baltimpex\Invoice\Import\АА-0093373\ТРАНС-БИЗНЕС РАМЕНСКОЕ ООО_1540_Счет на оплату № 1540 от 01 июля 2024 г.xlsx
---

... ЗАМЕНА ЦЕНЫ НА КОММЕНТАРИЙ ...

### Загрузка отфильтрованного и дополненного "комментариями" файла

In [141]:
with open(os.path.join(config['BASE_DIR'], 'config', 'new_services.txt'), 'r', encoding='utf-8') as file:
    data = file.readlines()

In [142]:
# Удаление пустых строк из новых сервисов

data = list(filter(lambda x: bool(x), map(lambda y: y.strip(), data)))
data

["Терминальная обработка|000000421|Терминальная обработка по приёму (40' контейнер; Груженый; Контейнерный поезд (КП))",
 'Линейный фрахтовый сбор (THC)|000000420|Линейный фрахтовый сбор (ТНС)',
 'Оформление релиза (конт.)|ТК-008755|Оформление релиза',
 "Комплекс услуг по прибытию 20' контейнера на ст. Раменское|ТК-008899|Комплекс услуг по прибытию 20' контейнера на ст. Раменское",
 'Распечатывание товаросопроводительных документов|ТК-009018|+',
 'Обслуживание груза|Т1115    |Обслуживание груза «с колес», доставленного на одном транспортном средстве при декларировании Заказчиком (участник ВЭД, представитель, и т.п.) при таможенном оформлении в режиме «Выпуск для внутреннего потребления» для клиента',
 'Фрахт (Море)|000000574|OCEAN FREIGHT',
 'DROP OFF FEE|ТК-008677|DROP OFF FEE',
 'Морской фрахт|ТК-006053|Морской фрахт , Порт_отпр, Порт_назн, Номер_контейнера, Тип_контейнера Номер_Коносамента, НДС, Договор',
 'Сбор средств по фрахту (погрузо-рагрузочные работы в порту)|ТК-008680|Сбор с

In [143]:
regex = r'.*\|.*\|.*'
for i in data:
    if re.fullmatch(regex, i):
        pass
    else:
        print(i)

In [144]:
def create_dataframe(data_list):
    # Преобразуем список в список списков, разделяя строки по символу |
    data_split = [item.split('|') for item in data_list]
    
    # Создаем DataFrame с соответствующими заголовками столбцов
    df = pd.DataFrame(data_split, columns=['service', 'code', 'comment'])
    
    return df

In [145]:
df = create_dataframe(data)

In [146]:
df.tail()

,service,code,comment
277,Дегазация подкарантинной продукции,000000493,Дегазация подкарантинной продукции в контейнерах
278,DROP OFF FEE,ТК-008677,+
279,ПРР груженый опасным грузом импортный контейнер,ТК-009177,+
280,ПРР груженого импортного контейнера,ТК-008422,Организация ПРР груженый импортный контейнер
281,"Перемещение контейнера, выгрузка/погрузка груз...",ТК-002702,+


In [147]:
# замена + (как в 1 в 1)

def replace_plus(raw):
    if raw['comment'] == '+':
        raw['comment'] = raw['service']
    return raw

In [148]:
df.apply(replace_plus, axis=1)
df.tail()

,service,code,comment
277,Дегазация подкарантинной продукции,000000493,Дегазация подкарантинной продукции в контейнерах
278,DROP OFF FEE,ТК-008677,DROP OFF FEE
279,ПРР груженый опасным грузом импортный контейнер,ТК-009177,ПРР груженый опасным грузом импортный контейнер
280,ПРР груженого импортного контейнера,ТК-008422,Организация ПРР груженый импортный контейнер
281,"Перемещение контейнера, выгрузка/погрузка груз...",ТК-002702,"Перемещение контейнера, выгрузка/погрузка груз..."


In [149]:
df = df.drop_duplicates().reset_index(drop=True)
df.tail()

,service,code,comment
233,выставление контейнера,Т0632,Выставление контейнера и перетарка паллетизиро...
234,Дегазация подкарантинной продукции,000000493,Дегазация подкарантинной продукции в контейнерах
235,ПРР груженый опасным грузом импортный контейнер,ТК-009177,ПРР груженый опасным грузом импортный контейнер
236,ПРР груженого импортного контейнера,ТК-008422,Организация ПРР груженый импортный контейнер
237,"Перемещение контейнера, выгрузка/погрузка груз...",ТК-002702,"Перемещение контейнера, выгрузка/погрузка груз..."


In [151]:
# порядок полей

df = df[['service', 'comment', 'code']]

In [152]:
df.to_excel(r'asdasd.xlsx')

### Проверка на похожие строки

In [132]:
for i in df['comment']:
    lst = list(dict.fromkeys(df['comment'].to_list()))
    lst.remove(i)
    dif = difflib.get_close_matches(i, lst, n=1)
    if dif:
        print(i)
        print(dif[0])
        print('----------------------')

Терминальная обработка по приёму (40' контейнер; Груженый; Контейнерный поезд (КП))
1 Терминальная обработка по приёму (40' контейнер; Груженый; Контейнерный поезд (КП))
----------------------
Линейный фрахтовый сбор (ТНС)
Линейный фрахтовый сбор (THC)
----------------------
Комплекс услуг по прибытию 20' контейнера на ст. Раменское
Комплекс услуг по прибытию 40' контейнера на ст. Раменское
----------------------
Распечатывание товаросопроводительных документов
Распечатка сопроводительной документации
----------------------
Агентское вознаграждение (конвертация) 5%
Агентское вознаграждение (конвертация) 5% (шт)
----------------------
Чек въезд на базу
Чек на въезд
----------------------
Услуги по оформлению релиз-ордеров, т/х
Услуги по оформлению документов 
----------------------
Организация хранения контейнера в порту
Организация ремонта контейнера
----------------------
Перемещение контейнера для целей осмотра с использованием инспекционно-досмотрового комплекса (ИДК)
Перемещение ко

### Дополнительно

In [99]:
grouped = df.groupby(["service"]).agg(com=('comment', lambda x: list(x))).reset_index()

In [105]:
grouped[grouped['com'].apply(len) > 1]

,service,com
23,Въезд на базу,"[Чек въезд на базу, Чек на въезд]"
29,"Выявление и идентификация грибов,нематод, вред...","[Выявление и идентификация грибов,нематод, вре..."
32,Документационные услуги,"[Документационные услуги при организации ТЭО, ..."
36,Доставка проб в лабораторию,"[Доставка проб в лабораторию по заключению, До..."
38,Железнодорожная перевозка,"[Железнодорожная доставка поезд , рейс, Железн..."
49,Комплекс терминальных услуг по прибытию контей...,[Комплекс терминальных услуг по прибытию конте...
56,Комплексная услуга по прибытию одного груженог...,[Комплексная услуга по прибытию одного гружено...
61,Лабораторная фитопатологическая экспертиза сре...,[Лабораторная фитопатологическая экспертиза ср...
64,Линейный фрахтовый сбор (THC),"[Линейный фрахтовый сбор (ТНС), Линейный фрахт..."
66,Морской фрахт,"[Морской фрахт , Порт_отпр, Порт_назн, Номер_к..."


In [106]:
df[df['service'] == 'Сверхнорм. использование контейнера']

,service,code,comment
136,Сверхнорм. использование контейнера,000000593,СНП
137,Сверхнорм. использование контейнера,000000593,СНП сутки
138,Сверхнорм. использование контейнера,000000593,СНП сутки и далее
